<a href="https://colab.research.google.com/github/leejineun/ai-class/blob/main/8_fish_classfication_%EC%88%98%EC%97%85%EC%9A%A9-Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d crowww/a-large-scale-fish-dataset

In [ ]:
!ls

In [ ]:
!unzip a-large-scale-fish-dataset.zip

In [ ]:
import cv2
import matplotlib.pyplot as plt
from os import listdir
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
image_dir = './Fish_Dataset/Fish_Dataset'

In [ ]:
for file in listdir(image_dir) : 
    print(file)

In [ ]:
images, labels = [], []
gt_images, gt_labels = [], []
for file in listdir(image_dir) : 
    try : 
        for img in listdir('{}/{}/{}'.format(image_dir, file, file)) : 
            images.append('{}/{}/{}/{}'.format(image_dir, file, file, img))
            labels.append(file)
        for img in listdir('{}/{}/{} GT'.format(image_dir, file, file)) : 
            gt_images.append('{}/{}/{} GT/{}'.format(image_dir, file, file, img))
            gt_labels.append(file)
    except Exception : 
        continue

In [ ]:
df = pd.DataFrame({'path':images, 'label':labels})

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
df['label']=df['label'].astype('category')

In [ ]:
print(df['label'].value_counts(), '\n{}\n'.format('-'*50))

In [ ]:
### 시각화
# 라벨별 이미지 수 확인
plt.figure(figsize=(16, 16))
sns.countplot(df['label'])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(15,8), constrained_layout=True)
ax=ax.flatten()

j=0

for i in df['label'].unique():
    
    ax[j].imshow(plt.imread(df[df['label']==i].iloc[np.random.randint(30),0]))
    ax[j].set_title(i)
    j=j+1

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 라벨 인코딩

In [ ]:
encoder = LabelEncoder()
encoder.fit(df['label'])
df['label'] = encoder.transform(df['label'])

In [ ]:
df['label']

In [ ]:
encoder.classes_

In [ ]:
dummy = pd.get_dummies(df['label'])

In [ ]:
df_merged = pd.merge(df, dummy, left_on=df.index, right_on=dummy.index)

In [ ]:
# train_test_spilit
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test = train_test_split(df_merged, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
X_train

In [ ]:
# 이미지 어그멘테이션 및 path로부터 이미지 읽기
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
train_image_gen = ImageDataGenerator(
    rotation_range=30,
    preprocessing_function=preprocess_input
)

In [ ]:
test_image_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
X_train_img = train_image_gen.flow_from_dataframe(dataframe=X_train[:5000], x_col='path', y_col=[0, 1, 2, 3, 4, 5, 6, 7, 8], class_mode='raw', batch_size=32) # label : categorical

X_val_img = test_image_gen.flow_from_dataframe(dataframe=X_train[5000:], x_col='path', y_col=[0, 1, 2, 3, 4, 5, 6, 7, 8], class_mode='raw', batch_size=32) # label : categorical

X_test_img = test_image_gen.flow_from_dataframe(dataframe=X_test, x_col='path', y_col=[0, 1, 2, 3, 4, 5, 6, 7, 8], class_mode='raw', batch_size=32) # label : categorical

In [ ]:
# 모델 작성 및 학습
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2

In [ ]:
X_train_img[0][0].shape

In [ ]:
base_model = MobileNetV2(include_top=False, input_shape=X_train_img[0][0].shape[1:]) # 256 256 3

base_model.trainable = False

model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
history = model.fit(X_train_img, epochs=30, validation_data=X_val_img, callbacks=[early_stopping])